# Dictionaries and Set
### Any running Python program has many dictionaries active at the same time, even if the user's program code doesn't explicitly use a dictionary

# Chapter brief outline:
- Common dictionary methods
- Special handling for missing keys
- Variation of dict in the standard library
- The set and frozenset types
- How hash tables work
- Implementations of hash tables:
    - Key type limitations, unpredictable ordering etc


# GENERIC Mapping type

MutableMapping
- Mapping
    - Container



In [1]:
from collections import abc

my_dict = {}
isinstance(my_dict, abc.Mapping)

True

# What is HASHABLE?
An object is hashable if it has a hash value that never changes during its lifetime
(`can be accomplished with a __hash__() method`)and
can be compared to other objects (`using an __eq__() method`)

The hashable objects using `__eq__()` must have the same hash value


hashable types:
- str
- bytes
- numeric types
- set (always hashable because its element must be hashable by definition)
- tuple: (only if all its items ara hashable)
- All of python's immutable built-in type (except those with ref to unHashable types)
- User defined types are hashable by default (because their hash value is their `id()`)


In [2]:
# Hashable tuples

tt = (1, 2, (30, 40))
hash(tt)

-3907003130834322577

In [3]:
tl = (1, 2, [30, 40]) # lists are not hashable

# Ways of implementing a dict:

In [4]:
a = dict(one=1, two=2, three=3)
a

{'one': 1, 'two': 2, 'three': 3}

In [5]:
b = {'one': 1, 'two': 2, 'three': 3}
b

{'one': 1, 'two': 2, 'three': 3}

In [6]:
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
c

{'one': 1, 'two': 2, 'three': 3}

In [7]:
d = dict([('two', 2), ('one', 1), ('three', 3)]) # with list of tuples
d

{'two': 2, 'one': 1, 'three': 3}

In [8]:
e = dict({'three': 3, 'one': 1, 'two': 2})
e

{'three': 3, 'one': 1, 'two': 2}

In [9]:
a == b == c == d == e

True

# DICT Comprehension:
# `{key:value for key, value in tuple }`

In [10]:
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (1, 'United States'),
    (62, 'Indonesia'),
    (55, 'Brazil'),
    (92, 'Pakistan'),
    (880, 'Bangladesh'),
    (234, 'Nigeria'),
    (7, 'Russia'),
    (81, 'Japan'),
]

country_code = {country: code for code, country in DIAL_CODES  }
country_code

{'China': 86,
 'India': 91,
 'United States': 1,
 'Indonesia': 62,
 'Brazil': 55,
 'Pakistan': 92,
 'Bangladesh': 880,
 'Nigeria': 234,
 'Russia': 7,
 'Japan': 81}

In [11]:
# generator

country_code = {code: country.upper() for country, code in country_code.items() if code < 66}
country_code

{1: 'UNITED STATES', 62: 'INDONESIA', 55: 'BRAZIL', 7: 'RUSSIA'}

In [12]:
country_code.items()

dict_items([(1, 'UNITED STATES'), (62, 'INDONESIA'), (55, 'BRAZIL'), (7, 'RUSSIA')])

In [13]:
country_code.items().__sizeof__()


24

# Handling missing keys with setdefault


In [14]:
"""Build an index mapping word -> list of occurrences """

import sys
import  re

WORD_RE = re.compile('\w+')

index = {}
with open('poem.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)

            # very ugly for a reason
            occurrences = index.get(word, [])
            occurrences.append(location)
            index[word] = occurrences

for word in sorted(index, key=str.upper):
    print(word, index[word])

FileNotFoundError: [Errno 2] No such file or directory: 'poem.txt'

In [ ]:
"""Better code"""

WORD_RE = re.compile('\w+')

index = {}
with open('poem.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            index.setdefault(word, []).append(location)

for word in sorted(index, key=str.upper):
    print(word, index[word])

In [ ]:
my_dict = dict(one=1, two=2, three=3, four=4)
my_dict.get("five", "Not Found")

In [ ]:
my_dict.setdefault("six", "Key not found!")

# Mappings with flexible key lookup

## Two approaches:
- Use a defaultdict instead of the plain dict
- Subclass dict and implement the `__missing__()` method

# defaultdict: Another take on missing keys

## How it works
i.e, given an empty defaultdict created as `dd = defaultdict(list)` if  `new-key` is not in dd
then the expression  `dd['new-key']` does the following:

- Calls `list()` to create a new list
- inserts the list into `dd` using `new-key` as key
- returns ref to that list

In [ ]:
import  collections

WORD_RE = re.compile('\w+')

index = collections.defaultdict(list)
with open('poem.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)

            index[word].append(location)


for word in sorted(index, key=str.upper):
    print(word, index[word])


# The `__missing__` Method:

The `__missing__` is invoked by the `__getitem__` method

In [ ]:
class StrKeyDict0(dict):

    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return  self[str(key)]

    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()


In [ ]:

d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d

In [ ]:
d['2']

In [ ]:
d[4]

In [ ]:
#d[1]

In [ ]:
d.get('2')

In [ ]:
d.get(4)

In [ ]:
d.get(1, 'N/A')

In [ ]:
2 in d


- StrKeyDict0 inherits from dict
- Check weather jey is already a str, If it is, and it's missing, raise KeyError.
- Build str from key and look it uo.
- The get method delegates to `__getitem__` by using the `self[key]` notation; that gives the opportunity for our
`__missing__` to act.
- If a KeyError was raised, `__missing__ already failed so, we return the default value.
- Search for unmodified key `(the instance may contain non-str keys)` then for a str built from the key.



# Variations of dict

- collections.OrderedDict
    - maintains, keys insertion order
- Collections.ChainMap
    - Holds a list of mappings which can be searched as one.
- Collections.Counter
- Collections.UserDict
    - Designed to be subclassed


In [ ]:
# Collections.ChainMap

import builtins
from collections import  ChainMap
pylookup = ChainMap(locals(), globals(), vars(builtins))


In [ ]:
# Counter
from collections import Counter

ct = collections.Counter('abracadabra')
ct

In [ ]:
ct.update('aaaaazzz')

In [ ]:
ct

In [ ]:
ct.most_common(2)

# Subclassing UserDict


In [ ]:
import collections

class StrKeyDict(collections.UserDict):

    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]

    def __contains__(self, key):
        return str(key) in self.data

    def __setitem__(self, key, value):
        self.data[str(key)] = value

In [ ]:
d = StrKeyDict()
d['name'] = 'kelvin'
d

# Immutable mappings
 The mapping types provided by the standard library are all mutable, but you may need to
 guarantee that a user cannot change  a mapping by mistake.
 # `MappingProxyType`.

creates a copy of the original data tha can be read only. updates make on the original
data can be seen on the new copy, but the new copy cannot update the original copy.
`basically a one way binding`.

In [ ]:
from types import MappingProxyType

d = {1: 'A'}
d_proxy = MappingProxyType(d)
d_proxy

In [ ]:
d_proxy[1]

In [ ]:
d[2] = 'B'

In [ ]:
d[2]

In [ ]:
# d_proxy[3] = "2" // d_proxy does not support item assignment

In [ ]:
d_proxy

# Set theory
A set is a collection of unique objects. A basic use case is removing duplications

Union: `a and b, a | b`

Intersection: `a & b`

Difference: `a - b`


In [ ]:
l = ['spam', 'spam', 'eggs', 'spam']
ll = ['spam', 'spam', 'eggs', 'spam', "me", "now"]
set(l)

In [ ]:
list(set(l))

In [ ]:
# Count occurrences of needles in a heystack
needles = {'new', 'now', 'cool', 'me'}
heystack = {'hey', 'hey', 'cool', 'me', 'me', 'cool', 'cool'}
l3 = list(needles & heystack)
count = len(needles & heystack)
count

In [ ]:
l = ['spam', 'spam', 'eggs', 'spam']
l2 = ['spam', 'spam', 'eggs', 'spam', "me", "now"]
count  = list(set(l) & set(l2))
# OR
count2 = list(set(l).intersection(set(l2)))
count2

# Set literals
`{1, 2, 3, 4, ....}`

# Set comprehension

In [ ]:
from unicodedata import name
spacial_chars =  {chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')}
print(spacial_chars)

# *`dict`* and *`set`* under the hood
---
### Some questions:
- How efficient is Python dict and set?
- Why are they unordered
- Why can't we use any Python object as a dict key or set element
- why does the order of the dict keys or set elements depend on insertion order
    and may change during the lifetime of the structure
- Why is it bad to add items to a dict or set while iterating through it